In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
os.getcwd()
print(os.getcwd())

c:\Users\zhouy217\OneDrive - Pfizer\Documents\datapipline_outline\Data-Pipeline


In [1]:
from langdetect import detect
language_detect_outcome=[]
for sen in sentences:
    sentence_language=detect(sen)
    language_detect_outcome.append(sentence_language)


'en'

In [7]:
detect("War doesn't show who's right, just who's left.")

'en'

In [2]:
detect("Ein, zwei, drei, vier")

'de'

In [3]:
detect("今天天气不错")

'ko'

In [4]:
detect("是面向极客的计算机科学门户")

'zh-cn'

In [6]:
detect("にほんご")

'ja'

In [5]:
from textblob import TextBlob
   
  
L = ["Geeksforgeeks is a computer science portal for geeks",
    "Geeksforgeeks - это компьютерный портал для гиков",
    "Geeksforgeeks es un portal informático para geeks",
    "Geeksforgeeks是面向极客的计算机科学门户",
    "Geeksforgeeks geeks के लिए एक कंप्यूटर विज्ञान पोर्टल है",
    "Geeksforgeeksは、ギーク向けのコンピューターサイエンスポータルです。",
    ]
  
for i in L:
      
    # Language Detection
    lang = TextBlob(i) 
    print(lang.detect_language())

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [1]:
from textblob import TextBlob
b = TextBlob("bonjour")
b.detect_language()

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [2]:

import pandas as pd # 读取文件
import numpy as np # 数值计算
import nagisa # 日文分词
from sklearn.feature_extraction.text import TfidfVectorizer # 文本特征提取
from sklearn.linear_model import LogisticRegression # 逻辑回归
from sklearn.pipeline import make_pipeline # 组合流水线

# 读取数据
train_cn = pd.read_excel('c:\\Users\\zhouy217\OneDrive - Pfizer\\Documents\\datapipline_outline\\Data-Pipeline\\zh_trian.xlsx')
train_ja = pd.read_excel('c:\\Users\\zhouy217\OneDrive - Pfizer\\Documents\\datapipline_outline\\Data-Pipeline\\ja_train.xlsx')
train_en = pd.read_excel('c:\\Users\\zhouy217\OneDrive - Pfizer\\Documents\\datapipline_outline\\Data-Pipeline\\en_train.xlsx')

test_ja = pd.read_excel('c:\\Users\\zhouy217\OneDrive - Pfizer\\Documents\\datapipline_outline\\Data-Pipeline\\testA.xlsx', sheet_name='日语_testA')
test_en = pd.read_excel('c:\\Users\\zhouy217\OneDrive - Pfizer\\Documents\\datapipline_outline\\Data-Pipeline\\testA.xlsx', sheet_name='英文_testA')

# 文本分词
train_ja['words'] = train_ja['原始文本'].apply(lambda x: ' '.join(nagisa.tagging(x).words))
train_en['words'] = train_en['原始文本'].apply(lambda x: x.lower())

test_ja['words'] = test_ja['原始文本'].apply(lambda x: ' '.join(nagisa.tagging(x).words))
test_en['words'] = test_en['原始文本'].apply(lambda x: x.lower())

# 训练TFIDF和逻辑回归
pipline = make_pipeline(
    TfidfVectorizer(),
    LogisticRegression()
)
pipline.fit(
    train_ja['words'].tolist() + train_en['words'].tolist(),
    train_ja['意图'].tolist() + train_en['意图'].tolist()
)

# 模型预测
test_ja['意图'] = pipline.predict(test_ja['words'])
test_en['意图'] = pipline.predict(test_en['words'])
test_en['槽值1'] = np.nan
test_en['槽值2'] = np.nan

test_ja['槽值1'] = np.nan
test_ja['槽值2'] = np.nan

# 写入提交文件
writer = pd.ExcelWriter('submit.xlsx')
test_en.drop(['words'], axis=1).to_excel(writer, sheet_name='英文_testA', index=None)
test_ja.drop(['words'], axis=1).to_excel(writer, sheet_name='日语_testA', index=None)
writer.save()
writer.close()


In [3]:
test_ja['意图'] 

0            open_ac_mode
1                 open_ac
2                 open_ac
3                 open_ac
4                 open_ac
              ...        
764    open_collect_music
765    play_collect_music
766    open_collect_music
767    play_collect_music
768    play_collect_music
Name: 意图, Length: 769, dtype: object

In [4]:
test_en['意图'] 

0                 open_ac
1            open_ac_mode
2                 open_ac
3                 open_ac
4                 open_ac
              ...        
517    play_collect_music
518    play_collect_music
519    play_collect_music
520    play_collect_music
521    play_collect_music
Name: 意图, Length: 522, dtype: object

In [7]:
import spacy
from spacy.lang.ja.examples import sentences 

nlp = spacy.load("ja_core_news_sm")
doc = nlp(sentences[0])
print(doc.text)
for token in doc:
    print(token.text, token.pos_, token.dep_)

アップルがイギリスの新興企業を１０億ドルで購入を検討
アップル NOUN nsubj
が ADP case
イギリス PROPN nmod
の ADP case
新興 NOUN compound
企業 NOUN obj
を ADP case
１０億 NUM compound
ドル NOUN obl
で ADP case
購入 NOUN obj
を ADP case
検討 NOUN ROOT


In [6]:
sentences

['アップルがイギリスの新興企業を１０億ドルで購入を検討',
 '自動運転車の損害賠償責任、自動車メーカーに一定の負担を求める',
 '歩道を走る自動配達ロボ、サンフランシスコ市が走行禁止を検討',
 'ロンドンはイギリスの大都市です。']

In [11]:
doc=nlp(sentences[0])
for word in doc.ents:
    print(word.text,word.label_)


アップル ORG
イギリス GPE
１０億ドル MONEY


In [10]:
doc = nlp("This is a sentence.")
ner = nlp.add_pipe("ner")
# This usually happens under the hood
processed = ner(doc)

ValueError: [E007] 'ner' already exists in pipeline. Existing names: ['tok2vec', 'parser', 'senter', 'ner', 'attribute_ruler']